# Week 6 Programming Assignment


In [1]:
import pandas as pd
import bqplot
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
import matplotlib.colors as mpl_colors

In [2]:
df = pd.read_csv('building_inventory.csv')
df.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided


In [3]:
num_cols = {}
i = 0
for name in list(df['Agency Name'].unique()):
    num_cols[name] = i
    i+=1
num_cols.keys()
list_columns = list(num_cols.keys())

In [4]:
df['Agency Name'].replace(list(num_cols.keys()),
                        (num_cols.values()), inplace=True)
df.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,0,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,0,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,0,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,0,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,0,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided


In [5]:
def generate_histogram_from_lat_long(df, nlong=35, nlat=17, longmin=0, longmax=34,
                                     latmin=1, latmax=17,
                                     takeLog=True):
    long_bins = np.linspace(longmin, longmax, nlong+1)
    lat_bins = np.linspace(latmin, latmax, nlat+1)
    hist2d, long_edges, lat_edges = np.histogram2d(df['Agency Name'], 
                                                   df['Congress Dist'],
                                                   weights=df['Square Footage'],
                                                  bins = [long_bins,lat_bins])
    hist2d = hist2d.T
    if takeLog:
        hist2d[hist2d <= 0] = np.nan 
    long_centers = (long_edges[:-1] + long_edges[1:]) / 2
    lat_centers = (lat_edges[:-1] + lat_edges[1:]) / 2
    return hist2d, long_centers, lat_centers, long_edges, lat_edges

In [6]:
hist2d, long_centers, lat_centers, long_edges, lat_edges = generate_histogram_from_lat_long(df)

In [7]:
import datetime as dt

In [8]:
i,j = 16,0

In [9]:
longs = [long_edges[j], long_edges[j+1]]
lats = [lat_edges[i],lat_edges[i+1]]

In [10]:
region_mask = ( (df['Congress Dist'] >= lats[0]) & (df['Congress Dist']<=lats[1]) &\
                (df['Agency Name'] >= longs[0]) & (df['Agency Name']<=longs[1]) )

lats, longs, df['Congress Dist'][region_mask]

([16.058823529411764, 17.0],
 [0.0, 0.9714285714285714],
 0       17
 1       17
 2       17
 3       17
 4       17
         ..
 8503    17
 8504    17
 8505    17
 8550    17
 8725    17
 Name: Congress Dist, Length: 306, dtype: int64)

In [11]:
mySelectedLabel = ipywidgets.Label()

In [12]:
col_sc = bqplot.ColorScale(scheme="BuPu", 
                           min=np.nanmin(hist2d), 
                           max=np.nanmax(hist2d))
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# (2) create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

x_ax = bqplot.Axis(scale = x_sc, label='Agency Name Key')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Congress Dist')

# (3) Marks
heat_map = bqplot.GridHeatMap(color = hist2d,
                              row = lat_centers, 
                              column = long_centers,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})


In [13]:
x_scl = bqplot.OrdinalScale()
y_scl = bqplot.LinearScale()

# (2) Axis
ax_xcl = bqplot.Axis(scale = x_scl, tick_rotate = 90, label = 'Year')
ax_ycl = bqplot.Axis(scale = y_scl, orientation = 'vertical', label = 'Total Square Footage')

# (3) Marks

i,j = 16,0
longs = [long_edges[j], long_edges[j+1]]
lats = [lat_edges[i],lat_edges[i+1]] 

fig_line = bqplot.Lines(x = [],
                               y = [],
                              scales={'x':x_scl, 'y':y_scl})

In [14]:
fig_line

Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': OrdinalScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9})

In [15]:
def get_data_value(change):
    if len(change['owner'].selected) == 1:
        i,j = change['owner'].selected[0]
        v = hist2d[i,j]
        mySelectedLabel.value = 'Total Square footage = ' + str(v)

        longs = [long_edges[j], long_edges[j+1]]
        lats = [lat_edges[i],lat_edges[i+1]]
        region_mask = ( (df['Congress Dist'] >= lats[0]) & (df['Congress Dist']<=lats[1]) &\
                    (df['Agency Name'] >= longs[0]) & (df['Agency Name']<=longs[1]) )
        fig_line.x = df['Year Acquired'][region_mask].sort_values(axis=0, ascending=True)
        fig_line.y = df['Square Footage'][region_mask]

heat_map.observe(get_data_value, 'selected')

In [16]:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])
fig_dur = bqplot.Figure(marks = [fig_line], axes = [ax_xcl, ax_ycl])

In [17]:
fig_heatmap.layout.min_width='400px'
fig_dur.layout.min_width='600px'

myDashboard = ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_heatmap,fig_dur])])
myDashboard